# Custom Set

Create a custom set type.

Sometimes it is necessary to define a custom data structure of some
type, like a set. In this exercise you will define your own set. How it
works internally doesn't matter, as long as it behaves like a set of
unique elements.

The tests require a constructor that takes an array. The internals of your custom set implementation can use other data structures but you may have to implement an outer constructor that takes exactly one array for the tests to pass.

Certain methods have a unicode operator equivalent. E.g. `intersect(CustomSet([1, 2, 3, 4]), CustomSet([]))` is equivalent to `CustomSet([1, 2, 3, 4]) ∩ CustomSet([])`.

## Version compatibility
This exercise has been tested on Julia versions >=1.0.

## Submitting Incomplete Solutions
It's possible to submit an incomplete solution so you can see how others have completed the exercise.

## Your solution

In [23]:
# submit
using Base
import Base.isempty, Base.union, Base.intersect, Base.union!, Base.intersect!, Base.in
import Base.issubset, Base.==, Base.length, Base.push!, Base.iterate, Base.copy


struct CustomSet{T} <: Base.AbstractSet{T}
    array::Array{T,1}
    b::Int
end

function CustomSet(a::Array)
    return CustomSet(sort(union(a, [])),0)
end

function isempty(set::CustomSet)
    return length(set.array) == 0
end
    
function intersect(s::CustomSet, t::CustomSet)
    return CustomSet(intersect(s.array, t.array))
end

    
function intersect!(s::CustomSet, t::CustomSet)
    sort!(intersect!(s.array, t.array))
end

function union(s::CustomSet, t::CustomSet)
    return CustomSet(union(s.array, t.array))
end

function union!(s::CustomSet, t::CustomSet)
    sort!(union!(s.array, t.array))
end

function issubset(s::CustomSet, t::CustomSet)
    return issubset(s.array, t.array)
end

function length(s::CustomSet)
    return length(s.array)
end

function disjoint(s::CustomSet, t::CustomSet)
    return length(intersect(s, t)) == 0
end

function ==(s::CustomSet, t::CustomSet)
    return s.array == t.array
end

function push!(s::CustomSet, e::Any)
    if !(e in s) 
        sort!(push!(s.array, e))
    end
end

function complement(superset::CustomSet, s::CustomSet)
    return CustomSet(setdiff(superset.array, s.array))
end
   
function complement!(superset::CustomSet, s::CustomSet)
    setdiff!(superset.array, s.array)
end
   
function in(e::Any, s::CustomSet)
    return e in s.array
end

function iterate(s::CustomSet, i::Int)
    return iterate(s.array, i)
end

function copy(s::CustomSet)
    return CustomSet(copy(s.array))
end

copy (generic function with 94 methods)

In [19]:
cs1 = CustomSet([1, 3])
cs2 = CustomSet([2, 3])
union!(cs1, cs2)
cs1

CustomSet{Any}(Any[1, 2, 3], 0)

## Test suite

In [20]:
# canonical data version: 1.3.0

using Test

# include("custom-set.jl")

# canonical data
@testset "empty" begin
    @test  isempty(CustomSet([]))
    @test !isempty(CustomSet([1]))
end

@testset "in (contains)" begin
    @test !(1 in CustomSet([]))
    @test   1 in CustomSet([1, 2, 3])
    @test !(4 in CustomSet([1, 2, 3]))
end

@testset "subset" begin
    @test  issubset(CustomSet([]), CustomSet([]))
    @test  issubset(CustomSet([]), CustomSet([1]))
    @test !issubset(CustomSet([1]), CustomSet([]))
    @test  issubset(CustomSet([1, 2, 3]), CustomSet([1, 2, 3]))
    @test  issubset(CustomSet([1, 2, 3]), CustomSet([4, 1, 2, 3]))
    @test !issubset(CustomSet([1, 2, 3]), CustomSet([4, 1, 3]))
end

@testset "disjoint" begin
    @test  disjoint(CustomSet([]), CustomSet([]))
    @test  disjoint(CustomSet([]), CustomSet([1]))
    @test  disjoint(CustomSet([1]), CustomSet([]))
    @test !disjoint(CustomSet([1, 2]), CustomSet([2, 3]))
    @test  disjoint(CustomSet([1, 2]), CustomSet([3, 4]))
end

@testset "equal" begin
    @test CustomSet([]) == CustomSet([])
    @test CustomSet([]) != CustomSet([1, 2, 3])
    @test CustomSet([1, 2, 3]) != CustomSet([])
    @test CustomSet([1, 2]) == CustomSet([2, 1])
    @test CustomSet([1, 2, 3]) != CustomSet([1, 2, 4])
    @test CustomSet([1, 2, 3]) != CustomSet([1, 2, 3, 4])
end

@testset "add" begin
    @test begin
        custom_set = CustomSet([])
        push!(custom_set, 3)
        custom_set == CustomSet([3])
    end
    @test begin
        custom_set = CustomSet([1, 2, 4])
        push!(custom_set, 3)
        custom_set == CustomSet([1, 2, 3, 4])
    end
    @test begin
        custom_set = CustomSet([1, 2, 3])
        push!(custom_set, 3)
        custom_set == CustomSet([1, 2, 3])
    end
end

@testset "intersection" begin
    @testset "in-place" begin
        @test begin
            cs1 = CustomSet([])
            cs2 = CustomSet([])
            intersect!(cs1, cs2)
            isempty(cs1)
        end
        @test begin
            cs1 = CustomSet([])
            cs2 = CustomSet([3, 2, 5])
            intersect!(cs1, cs2)
            isempty(cs1)
        end
        @test begin
            cs1 = CustomSet([1, 2, 3, 4])
            cs2 = CustomSet([])
            intersect!(cs1, cs2)
            isempty(cs1)
        end
        @test begin
            cs1 = CustomSet([1, 2, 3])
            cs2 = CustomSet([4, 5, 6])
            intersect!(cs1, cs2)
            isempty(cs1)
        end
        @test begin
            cs1 = CustomSet([1, 2, 3, 4])
            cs2 = CustomSet([3, 2, 5])
            intersect!(cs1, cs2)
            cs1 == CustomSet([2, 3])
        end
    end
    @testset "not in-place" begin
        @test isempty(intersect(CustomSet([]), CustomSet([])))
        @test isempty(intersect(CustomSet([]), CustomSet([3, 2, 5])))
        @test isempty(intersect(CustomSet([1, 2, 3, 4]), CustomSet([])))
        @test isempty(intersect(CustomSet([1, 2, 3]), CustomSet([4, 5, 6])))
        @test intersect(CustomSet([1, 2, 3, 4]), CustomSet([3, 2, 5])) == CustomSet([2, 3])
    end
end

@testset "complement (difference)" begin
    @testset "in-place" begin
        @test begin
            cs1 = CustomSet([])
            cs2 = CustomSet([])
            complement!(cs1, cs2)
            isempty(cs1)
        end
        @test begin
            cs1 = CustomSet([])
            cs2 = CustomSet([3, 2, 5])
            complement!(cs1, cs2)
            isempty(cs1)
        end
        @test begin
            cs1 = CustomSet([1, 2, 3, 4])
            cs2 = CustomSet([])
            complement!(cs1, cs2)
            cs1 == CustomSet([1, 2, 3, 4])
        end
        @test begin
            cs1 = CustomSet([3, 2, 1])
            cs2 = CustomSet([2, 4])
            complement!(cs1, cs2)
            cs1 == CustomSet([1, 3])
        end
    end
    @testset "not in-place" begin
        @test isempty(complement(CustomSet([]), CustomSet([])))
        @test isempty(complement(CustomSet([]), CustomSet([3, 2, 5])))
        @test complement(CustomSet([1, 2, 3, 4]), CustomSet([])) == CustomSet([1, 2, 3, 4])
        @test complement(CustomSet([3, 2, 1]), CustomSet([2, 4])) == CustomSet([1, 3])
    end
end

@testset "union" begin
    @testset "in-place" begin
        @test begin
            cs1 = CustomSet([])
            cs2 = CustomSet([])
            union!(cs1, cs2)
            isempty(cs1)
        end
        @test begin
            cs1 = CustomSet([])
            cs2 = CustomSet([2])
            union!(cs1, cs2)
            cs1 == CustomSet([2])
        end
        @test begin
            cs1 = CustomSet([1, 3])
            cs2 = CustomSet([])
            union!(cs1, cs2)
            cs1 == CustomSet([1, 3])
        end
        @test begin
            cs1 = CustomSet([1, 3])
            cs2 = CustomSet([2, 3])
            union!(cs1, cs2)
            cs1 == CustomSet([3, 2, 1])
        end
    end
    @testset "not in-place" begin
        @test isempty(union(CustomSet([]), CustomSet([])))
        @test union(CustomSet([]), CustomSet([2])) == CustomSet([2])
        @test union(CustomSet([1, 3]), CustomSet([])) == CustomSet([1, 3])
        @test union(CustomSet([1, 3]), CustomSet([2, 3])) == CustomSet([3, 2, 1])
    end
end

# language specific tests
@testset "implements correct abstract type" begin
    @test CustomSet <: Base.AbstractSet
end

@testset "length" begin
    @test length(CustomSet([])) == 0
    @test length(CustomSet([1, 2, 3])) == 3
end

@testset "iterable" begin
    @test begin
        cs1 = CustomSet([1, 2, 3, 4])
        cs2 = CustomSet([])
        for element in cs1
            push!(cs2, element)
        end
        cs1 == cs2
    end
end

@testset "copy" begin
    @test begin
        cs1 = CustomSet([1, 2, 3])
        cs2 = copy(cs1)
        push!(cs1, 4)
        cs2 == CustomSet([1, 2, 3])
    end
end

Test Summary: | Pass  Total
empty         |    2      2
Test Summary: | Pass  Total
in (contains) |    3      3
Test Summary: | Pass  Total
subset        |    6      6
Test Summary: | Pass  Total
disjoint      |    5      5
Test Summary: | Pass  Total
equal         |    6      6
Test Summary: | Pass  Total
add           |    3      3
Test Summary: | Pass  Total
intersection  |   10     10
Test Summary:           | Pass  Total
complement (difference) |    8      8
Test Summary: | Pass  Total
union         |    8      8
Test Summary:                    | Pass  Total
implements correct abstract type |    1      1
Test Summary: | Pass  Total
length        |    2      2
Test Summary: | Pass  Total
iterable      |    1      1
Test Summary: | Pass  Total
copy          |    1      1


Test.DefaultTestSet("copy", Any[], 1, false)

## Prepare submission
To submit your exercise, you need to save your solution in a file called `custom-set.jl` before using the CLI.
You can either create it manually or use the following functions, which will automatically write every notebook cell that starts with `# submit` to the file `custom-set.jl`.


In [24]:
# using Pkg; Pkg.add("Exercism")
using Exercism
Exercism.create_submission("custom-set")

1630